In [ ]:
import socket
import threading

# 用于存储客户端的连接信息和发送的最后一个包的序列号
clients = {}

def handle_client(client_socket, client_address):
    global clients
    sequence_number = 0
    while True:
        try:
            data, _ = client_socket.recvfrom(1024)
            message = data.decode()
            # 解析消息中的序列号和数据
            seq, _, msg = message.partition(':')
            seq = int(seq)
            
            # 如果收到的是ACK，则更新序列号
            if msg == 'ACK':
                clients[client_address]['last_acked'] = seq
            else:
                # 发送ACK
                client_socket.sendto(f"{seq}:ACK".encode(), client_address)
                print(f"Sent ACK {seq} to {client_address}")
        except Exception as e:
            print(f"Error: {e}")
            break

def server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind(('localhost', 12345))
    print("UDP Server listening on port 12345")

    while True:
        data, client_address = server_socket.recvfrom(1024)
        if client_address not in clients:
            clients[client_address] = {'socket': socket.socket(socket.AF_INET, socket.SOCK_DGRAM), 'last_acked': 0}
            thread = threading.Thread(target=handle_client, args=(clients[client_address]['socket'], client_address))
            thread.start()
        else:
            handle_client(clients[client_address]['socket'], client_address)

if __name__ == "__main__":
    server()

In [2]:
#client
import sys
import socket
import time
import threading
import random
import numpy as np
from tqdm import tqdm

#define
HOST="127.0.0.1"
SERVER_RECEIVE_PORT=50000 #服务端接收报文
SERVER_ACK_PORT=50001 #服务端发送ACK 非必须
CLIENT_SEND_PORT=50002 #客户端发送报文 非必须
CLIENT_ACK_PORT=50003 #客户端接收ACK
MAX_DATA_LENGTH=4194304 #最大报文长度4M
SWND=8 #发送窗口
TIMEOUT=2 #超时重传时间

def msg_sender(msg_array,ack_array):
    #信息发送套接字
    m_sender=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    m_sender.bind((HOST,CLIENT_SEND_PORT))

    time_array=np.zeros_like(msg_array,dtype=float)
    segment_p=0
    swnd_p=0
    with tqdm(total=len(msg_array)) as pbar:
        while(True):
            if(segment_p==len(msg_array)):#发送完成
                pbar.update(SWND)
                m_sender.close()
                break
            while ack_array[swnd_p]:#移动窗口
                swnd_p+=1
                pbar.update()
            if segment_p==swnd_p+SWND:
                for _ in range(segment_p-SWND,segment_p):
                    if ack_array[_]==False and time.perf_counter()-time_array[_]>TIMEOUT:
                        m_sender.sendto(str(msg_array[_]).encode("utf8"),(HOST,SERVER_RECEIVE_PORT))
                continue
            m_sender.sendto(str(msg_array[segment_p]).encode("utf8"),(HOST,SERVER_RECEIVE_PORT))
            time_array[segment_p]=time.perf_counter()
            segment_p+=1


def ack_receiver(ack_array):
    #ack接收套接字
    ack_receiver=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    ack_receiver.bind((HOST,CLIENT_ACK_PORT))
    ack_receiver.settimeout(2*TIMEOUT)
    while(True):
        ack,addr=ack_receiver.recvfrom(1024)
        if ack:
            ack=int(ack.decode("utf8"))
            ack_array[ack]=True

class senderThread(threading.Thread):
    def __init__(self,msg_array,ack_array):
        threading.Thread.__init__(self)
        self.msg_array=msg_array
        self.ack_array=ack_array
    def run(self):
        msg_sender(self.msg_array,self.ack_array)

class receiverThread(threading.Thread):
    def __init__(self,ack_array):
        threading.Thread.__init__(self)
        self.ack_array=ack_array
    def run(self):
        ack_receiver(self.ack_array)


if __name__=="__main__":
    msg_length=128
    massage=np.zeros([msg_length],dtype=int)
    for _ in range(len(massage)):
        massage[_]=_
    ack=np.zeros([msg_length],dtype=bool)
    #print(massage.shape,ack.shape)
    thread1=senderThread(massage,ack)
    thread2=receiverThread(ack)
    thread1.start()
    thread2.start()

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/home/xiongshihao/anaconda3/envs/test/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  0%|          | 0/128 [00:00<?, ?it/s]    self.run()
  File "/tmp/ipykernel_2216253/3230257381.py", line 75, in run


  File "/tmp/ipykernel_2216253/3230257381.py", line 50, in ack_receiver
OSError: [Errno 98] Address already in use


In [ ]:
#server
import socket
import threading
import queue
import time


#define
HOST="127.0.0.1"
SERVER_RECEIVE_PORT=50000 #服务端接收报文
SERVER_ACK_PORT=50001 #服务端发送ACK
CLIENT_SEND_PORT=50002 #客户端发送报文
CLIENT_ACK_PORT=50003 #客户端接收ACK
MAX_DATA_LENGTH=4194304 #最大报文长度4M
SWND=8 #发送窗口
TIMEOUT=2 #超时重传时间

def msg_receiver(rec_queue):
    #接收发送端报文，存入rec_array
    rec_socket=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    rec_socket.bind((HOST,SERVER_RECEIVE_PORT))
    rec_socket.settimeout(2*TIMEOUT)
    while(True):
        msg,addr=rec_socket.recvfrom(1024)
        #print(msg.decode("utf8"),addr)
        msg=msg.decode("utf8")
        if msg:
            print(msg)
            if not rec_queue.full():
                rec_queue.put(msg)
            continue

def ack_sender(rec_queue):
    #回复ack信息，线程间通过队列通信
    ack_socket=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    ack_socket.bind((HOST,SERVER_ACK_PORT))
    wc_dog=time.perf_counter()
    while(True):
        if not rec_queue.empty():
            ack=str(rec_queue.get()).encode("utf8")
            ack_socket.sendto(ack,(HOST,CLIENT_ACK_PORT))
            wc_dog=time.perf_counter()
            continue
        if time.perf_counter()-wc_dog>2*TIMEOUT:
            ack_socket.close()
            print("ACK TIMEOUT CLOSE")
            break


class msgrecThread(threading.Thread):
    def __init__(self,rec_queue):
        threading.Thread.__init__(self)
        self.rec_queue=rec_queue
    def run(self):
        msg_receiver(self.rec_queue)

class acksendThread(threading.Thread):
    def __init__(self,rec_queue):
        threading.Thread.__init__(self)
        self.rec_queue=rec_queue
    def run(self):
        ack_sender(self.rec_queue)


if __name__=="__main__":
    msg_queue=queue.Queue(MAX_DATA_LENGTH)
    thread1=msgrecThread(msg_queue)
    thread2=acksendThread(msg_queue)
    thread1.start()
    thread2.start()

0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8


In [11]:
import time
from tqdm import tqdm

with tqdm(total=100) as pbar:
    for i in range(100):
        time.sleep(0.02)
        pbar.update(1)

100%|██████████| 100/100 [00:02<00:00, 49.15it/s]
